In [140]:
import requests
import alpaca_trade_api as alpaca
import math

API_KEY = 'PKJ7V71GWHKKT0SQ66C2'
SECRET_KEY = '5dqPh7yylDgSNrGMKcesUDiQMwRmUg0fe1XcQfEi'

HEADERS = {'APCA-API-KEY-ID': API_KEY,
           'APCA-API-SECRET-KEY': SECRET_KEY}

ALPACA_BASE_URL = 'https://paper-api.alpaca.markets'
DATA_URL = 'https://data.alpaca.markets'
rest_api = alpaca.REST(API_KEY, SECRET_KEY, ALPACA_BASE_URL)
investment = 1000
min_profit_pct = 0.03

coins = ['AVAX', 'BCH', 'DOGE', 'ETH', 'LINK', 'LTC', 'MATIC', 'SOL', 'SUSHI', 'UNI', 'YFI']

In [210]:
# initialize price dictionary
# def init_coin_pair_dict(coins: list) -> list:
#     full_list = []
#     for coin in coins:
#         full_list.append(coin+'/BTC')
#         full_list.append(coin + '/USD')
#     price_dict = dict.fromkeys(full_list, 0)
#     return price_dict
# coins = ['AVAX', 'BCH', 'DOGE', 'ETH', 'LINK', 'LTC', 'MATIC', 'SOL', 'SUSHI', 'UNI', 'YFI']
# prices_dict = init_coin_pair_dict(coins)

def _init_coin_pair_dict(coins: list):
    # create list with symbol based in btc & usd
    full_list = []
    for coin in coins:
        full_list.append(coin+'/BTC')
        full_list.append(coin + '/USD')
    full_list = ",".join(full_list)
    full_list += ",BTC/USD"
    # initiate dictionary for prices
    prices_dict = dict.fromkeys(coins, (0, 0)) # tuple to store (BTC/ USD)
    return prices_dict, full_list

def get_latest_quote(coins) -> dict:
    """
    :param coins: list of coins that have btc denominated pair
    :return: {'COIN': (COIN-BTC px, COIN-USD px)}
    """
    prices_dict, full_list = _init_coin_pair_dict(coins)
    quote_dict = get_quote(full_list)
    # try:
    #     quote = requests.get('{0}/v1beta2/crypto/latest/trades?symbols={1}'
    #                          .format(DATA_URL, full_list), headers=HEADERS)
    # except Exception as e:
    #     print(f"ERROR when fetching Alpaca data: {e}")
    #     return False
    # quote_dict = quote.json()['trades']
    # fill prices_dict with quote prices
    for coin in coins:
        to_btc = quote_dict[f"{coin}/BTC"]['p']
        to_usd = quote_dict[f"{coin}/USD"]['p']
        prices_dict[coin] = (to_btc, to_usd)
    prices_dict['BTC/USD'] = quote_dict[f"BTC/USD"]['p']
    return prices_dict

In [216]:
get_quote('BTC/USD')

{'BTC/USD': {'i': 46970056,
  'p': 19132,
  's': 0.0014,
  't': '2022-10-15T16:12:37.060878Z',
  'tks': 'B'}}

In [221]:
def get_account_balance():
    """ place order to Alpaca """
    try:
        balance = requests.get(
            '{0}/v2/account'.format(ALPACA_BASE_URL), headers=HEADERS)
        return balance.json()['portfolio_value']
    except Exception as e:
        print(f"ERROR when getting account balance: {e}")
        return False
get_account_balance()

'98493.28535131177366284'

In [223]:
def round_down_decimal(number, decimal):
    return math.floor(number * pow(10, decimal))/pow(10, decimal)

def get_account_balance():
    """ place order to Alpaca """
    try:
        balance = requests.get(
            '{0}/v2/account'.format(ALPACA_BASE_URL), headers=HEADERS)
        return round(float(balance.json()['portfolio_value']),2)
    except Exception as e:
        print(f"ERROR when getting account balance: {e}")
        return False

def get_quote(symbol):
    """ get latest price of symbol """
    try:
        quote = requests.get('{0}/v1beta2/crypto/latest/trades?symbols={1}'
                             .format(DATA_URL, symbol), headers=HEADERS)
        return quote.json()['trades']
    except Exception as e:
        print(f"ERROR when fetching symbol price: {e}")
        return False

def liquidate_positions():
    """ liquidate all positions """
    try:
        return requests.delete(
            '{0}/v2/positions'.format(ALPACA_BASE_URL), headers=HEADERS)
    except Exception as e:
        print(f"ERROR when liquidating positions from Alpaca: {e}")
        return False

def get_position(symbol):
    """ get position of symbol
    :param symbol: in form of 'BTCUSD'
    :return: position value
    """
    try:
        position = requests.get(
            '{0}/v2/positions/{1}'.format(ALPACA_BASE_URL, symbol), headers=HEADERS)
        position = float(position.json()['qty'])
        # qty = round_down_decimal(position,2)
        return position
    except Exception as e:
        print(f"ERROR when getting position from Alpaca: {e}")
        return False

def place_order(symbol, qty, side):
    """ place order to Alpaca """
    try:
        order = requests.post(
            '{0}/v2/orders'.format(ALPACA_BASE_URL), headers=HEADERS, json={
                'symbol': symbol,
                'qty': qty,
                'side': side,
                'type': 'market',
                'time_in_force': 'gtc',
            })
        return order
    except Exception as e:
        print(f"ERROR when placing order to Alpaca: {e}")
        return False

def check_arbitrage_opp(coin, prices_tuple, btc_usd):
    """
    :param prices_tuple: (COIN-BTC px, COIN-USD px)
    :param btc_usd: latest price of BTC/USD
    :return:
    """
    global investment
    global min_profit_pct
    coin_btc, coin_usd = prices_tuple
    threshold = coin_usd / btc_usd
    spread = abs(threshold - coin_btc)

    # BTC/USD is cheaper: buy BTC, convert to COIN, sell COIN
    if threshold > coin_btc * (1 + min_profit_pct / 100):
        print("Trade Triggered: BUY/BUY/SELL")
        print(f"Spread: {spread * 100:.5f}")
        buy_btc_qty = investment / btc_usd
        order_1 = place_order("BTC/USD", buy_btc_qty, "buy")
        if order_1.status_code==200:
            print(f"BUY BTC/USD:{buy_btc_qty:,.2} @ {btc_usd:,}")
            # TODO: for loop for highest available divider (0.95 down to 0.85)
            latest_coin_btc = get_quote(f"{coin}/BTC")[f"{coin}/BTC"]['p']
            sell_qty = get_position("BTCUSD") / (latest_coin_btc/0.95)  #(coin_btc/0.95)
            sell_qty = round_down_decimal(sell_qty, 2)
            order_2 = place_order(f"{coin}/BTC", sell_qty, "buy")
            if order_2.status_code==200:
                print(f"Buy {coin}/BTC:{sell_qty:,.2} @ {coin_btc:,.6f}")
                sell_qty_3 = get_position("BTCUSD")
                order_3 = place_order(f"{coin}/USD", sell_qty_3, "sell")
                if order_3.status_code==200:
                    print(f"Sell {coin}/USD:{sell_qty:,.2} @ {float(coin_usd):,.2}")
                    print("--------------------------------")
                else:
                    place_order(f"{coin}/BTC", sell_qty, "sell")
                    print(f"ERROR when selling {coin}/USD (3rd Trade)")
                    print("--------------------------------")
            else:
                print(f"ERROR when buying {coin}/BTC (2nd Trade)")
                print("--------------------------------")
        else:
            print("ERROR when buying BTC/USD (1st Trade)")
            print("--------------------------------")
        liquidate_positions()
    # COIN/USD is cheaper: buy COIN, convert to BTC, sell BTC
    if threshold < coin_btc * (1 - min_profit_pct / 100):
        print("Trade Triggered: BUY/SELL/SELL")
        print(f"Spread: {spread * 100:.5f}")
        buy_coin_qty = investment / coin_usd
        order_1 = place_order(f"{coin}/USD", buy_coin_qty, "buy")
        if order_1.status_code==200:
            print(f"Buy {coin}/USD:{buy_coin_qty:,.2f} @ {float(coin_usd):,.2f}")
            sell_qty_2 = get_position(f"{coin}USD")
            order_2 = place_order(f"{coin}/BTC", sell_qty_2, "sell")
            if order_2.status_code==200:
                print(f"Sell {coin}/BTC:{sell_qty_2:,.2f} @ {coin_btc:,.6f}")
                sell_qty_3 = get_position("BTCUSD")
                order_3 = place_order(f"BTC/USD", sell_qty_3, "sell")
                if order_3.status_code == 200:
                    print(f"Sell BTC/USD:{sell_qty_3:,.2f} @ {btc_usd:,}")
                    print("--------------------------------")
                else:
                    # _place_order(f"{coin}/BTC", sell_qty, "buy")
                    print("ERROR when selling BTC/USD (3rd Trade)")
                    print("--------------------------------")
            else:
                print(f"ERROR when selling {coin}/BTC (2nd Trade)")
                print("--------------------------------")
        else:
            print(F"ERROR when buying {coin}/USD (1st Trade)")
            print("--------------------------------")
        liquidate_positions()
    else:
        print(f"No arbitrage opportunity, spread: {spread:.5f}")
        print("--------------------------------")

def main():
    initial_balance = get_account_balance()
    prices_dict = get_latest_quote(coins)
    btc_usd = prices_dict['BTC/USD']
    for coin, prices_tuple in list(prices_dict.items())[:-1]:
        check_arbitrage_opp(coin, prices_tuple, btc_usd)
    ending_balance = get_account_balance()
    print(f"Position change: {ending_balance - initial_balance:2f}")

if __name__ == "__main__":
    main()

Trade Triggered: BUY/BUY/SELL
Spread: 0.00017
BUY BTC/USD:0.052 @ 19,159
ERROR when buying AVAX/BTC (2nd Trade)
--------------------------------
No arbitrage opportunity, spread: 0.00000
--------------------------------
Trade Triggered: BUY/BUY/SELL
Spread: 0.00093
BUY BTC/USD:0.052 @ 19,159
Buy BCH/BTC:8.7 @ 0.005694
Sell BCH/USD:8.7 @ 1.1e+02
--------------------------------
No arbitrage opportunity, spread: 0.00001
--------------------------------
Trade Triggered: BUY/BUY/SELL
Spread: 0.00000
BUY BTC/USD:0.052 @ 19,159
ERROR when buying DOGE/BTC (2nd Trade)
--------------------------------
No arbitrage opportunity, spread: 0.00000
--------------------------------
No arbitrage opportunity, spread: 0.00001
--------------------------------
Trade Triggered: BUY/SELL/SELL
Spread: 0.00003
Buy LINK/USD:143.83 @ 6.95
Sell LINK/BTC:143.47 @ 0.000363
Sell BTC/USD:0.05 @ 19,159
--------------------------------
Trade Triggered: BUY/SELL/SELL
Spread: 0.00040
Buy LTC/USD:19.42 @ 51.48
Sell LTC/BT

In [224]:
from alpaca_utils import *
get_all_positions()

[{'asset_id': '64bbff51-59d6-4b3c-9351-13ad85e3c752',
  'symbol': 'BTCUSD',
  'exchange': 'FTXU',
  'asset_class': 'crypto',
  'asset_marginable': False,
  'qty': '2.6040224543319414',
  'avg_entry_price': '19105.177993531502529',
  'side': 'long',
  'market_value': '49807.1374840070431578',
  'cost_basis': '49750.312489164498872323035807971296397',
  'unrealized_pl': '56.824994842544285476964192028703603',
  'unrealized_plpc': '0.0011422037772109',
  'unrealized_intraday_pl': '56.8249948425442855041759404201994',
  'unrealized_intraday_plpc': '0.0011422037772109',
  'current_price': '19127',
  'lastday_price': '19171',
  'change_today': '-0.0022951332742163',
  'qty_available': '2.6040224543319414'}]